# NCI WeatherBench-3b: Train a CNN

In this notebook we will go through all the steps required to train a fully convolutional neural network. Because this takes a while and uses a lot of GPU RAM a separate command line script (`train_nn.py`) is also provided in the `src` directory.

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import os
warnings.filterwarnings('ignore')
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
from score import *
import numpy as np
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dropout, Conv2D, Lambda, LeakyReLU
from tensorflow.python.client import device_lib
from dask.diagnostics import ProgressBar
from datetime import datetime
import train_nn
print (f"\nTF Device: {tf.config.list_physical_devices('GPU')}" )
train_nn.limit_mem()
from dask.distributed import Client

2024-03-26 23:56:07.078702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



TF Device: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-03-26 23:56:15.426545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0


In [2]:
client = Client(n_workers=12, threads_per_worker=1)  
client

/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.events-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.wa

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40319,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:37139,Total threads: 1
Dashboard: /proxy/45575/status,Memory: 7.92 GiB
Nanny: tcp://127.0.0.1:44249,


# Dataset: '5.625'

In [3]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

res     = '5.625'
datadir = f'/g/data/wb00/NCI-Weatherbench/{res}deg' 
print ("Data loading..." )

years = list(range(1999, 2022+1))
print (years)
z_files = [ file for year in years for file in glob.glob (fr'{datadir}/geopotential/*{year}*')  ] 
t_files = [ file for year in years for file in glob.glob (fr'{datadir}/temperature/*{year}*')    ] 

z = xr.open_mfdataset(z_files, combine='by_coords', parallel=True, chunks={'time': 10}).z.sel(level=[500]).load() 
t = xr.open_mfdataset(t_files, combine='by_coords', parallel=True, chunks={'time': 10}).t.sel(level=[850]).drop('level').load() 
datasets = [z, t]
print ("Merging ... ")
ds = xr.merge(datasets).compute()  

[2024-03-26 23:56:17]
Data loading...
[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Merging ... 
CPU times: user 1min 45s, sys: 12.9 s, total: 1min 58s
Wall time: 2min 20s


In [4]:
z['time'][:10]

<xarray.DataArray 'time' (time: 10)>
array(['1999-01-01T00:00:00.000000000', '1999-01-01T01:00:00.000000000',
       '1999-01-01T02:00:00.000000000', '1999-01-01T03:00:00.000000000',
       '1999-01-01T04:00:00.000000000', '1999-01-01T05:00:00.000000000',
       '1999-01-01T06:00:00.000000000', '1999-01-01T07:00:00.000000000',
       '1999-01-01T08:00:00.000000000', '1999-01-01T09:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1999-01-01 ... 1999-01-01T09:00:00

In [5]:
z['time'][-10:]

<xarray.DataArray 'time' (time: 10)>
array(['2022-12-31T14:00:00.000000000', '2022-12-31T15:00:00.000000000',
       '2022-12-31T16:00:00.000000000', '2022-12-31T17:00:00.000000000',
       '2022-12-31T18:00:00.000000000', '2022-12-31T19:00:00.000000000',
       '2022-12-31T20:00:00.000000000', '2022-12-31T21:00:00.000000000',
       '2022-12-31T22:00:00.000000000', '2022-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-12-31T14:00:00 ... 2022-12-31T23:00:00

In [6]:
def data_generate(ds, lead_time):
    ds_train = ds.sel(time=slice(*train_years))
    ds_valid = ds.sel(time=slice(*valid_years))
    ds_test  = ds.sel(time=slice(*test_years))

    print ("Data generation ... ")
    dic = {var: 500 for var in vars}
    dg_train = train_nn.DataGenerator(ds_train, dic, lead_time, batch_size=batch_size)
    dg_valid = train_nn.DataGenerator(ds_valid, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)
    dg_test =  train_nn.DataGenerator(ds_test, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)

    print(f'Mean = {dg_train.mean}; Std = {dg_train.std}')
    return dg_train, dg_valid, dg_test

def train(dg_train, dg_valid, dg_test, model_save_fn):
    print ("Train model ... ") 
    model = train_nn.build_cnn(filters, kernels, input_shape=(32, 64, len(vars)), dr=dr)    
    model.compile(keras.optimizers.Adam(lr), 'mse')
    print(model.summary())
    
    model.fit(dg_train, epochs=100, validation_data=dg_valid,
                          callbacks=[tf.keras.callbacks.EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=patience,
                          verbose=1,
                          mode='auto'
                      )]
                      )
    print(f'Saving model weights: {model_save_fn}')
    model.save_weights(model_save_fn)   
    return model

def prediction(model, dg_test, iterative, iterative_lead_time, pred_save_fn):    
    print("Create predictions ...")
    pred = train_nn.create_iterative_predictions(model, dg_test, iterative_lead_time) if iterative \
                                                        else train_nn.create_predictions(model, dg_test)
    print(f'Saving predictions: {pred_save_fn}')
    pred.to_netcdf(pred_save_fn)
    return pred
    
def evaluate(pred, iterative, test_years):     
    print("Evaluating forecast ...")
    valid_years = list( range(int(test_years[0]), int(test_years[1])+1 ))
    print ('all test_years:', valid_years)
    z500_valid_files = [ file for year in valid_years for file in glob.glob (fr'{datadir}/geopotential/*{year}*') ] 
    t850_valid_files = [ file for year in valid_years for file in glob.glob (fr'{datadir}/temperature/*{year}*')  ]     

    z500_valid = train_nn.load_test_data(z500_valid_files, 'z', slice(*test_years)) 
    t850_valid = train_nn.load_test_data(t850_valid_files, 't', slice(*test_years))     
    valid      = xr.merge([z500_valid, t850_valid], compat='override').compute()
    
    print(train_nn.evaluate_iterative_forecast(pred, valid, compute_weighted_rmse).load() if iterative \
                                                else train_nn.compute_weighted_rmse(pred, valid).compute())

def reset():
    print("Resetting ...")
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True)
    IPython.Application.instance().kernel.do_shutdown(True)


In [7]:
test_years =('2021', '2022')
test_years = list (range(int (test_years[0]), int(test_years[1])+1 ))
print ('test_years:', test_years)

test_years: [2021, 2022]


# 72 hours (3 days)

In [8]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
print (60*"-")
train_years=('1999', '2015')
valid_years=('2016', '2020')
test_years =('2021', '2022')
batch_size = 32
vars = ('z', 't')
lead_time = 72
filters = [64, 64, 64, 64, 2]
kernels = [5, 5, 5, 5, 5]
dr = 0
patience = 3
save_prefix = 'NCI_tutorial' 
print ('save_prefix:', save_prefix)
model_save_fn = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_cnn_3d.h5'
pred_save_fn  = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_cnn_3d.nc'
lr = 1e-4
iterative = False
iterative_lead_time = None

[2024-03-26 23:58:38]
------------------------------------------------------------
save_prefix: NCI_tutorial
CPU times: user 159 µs, sys: 0 ns, total: 159 µs
Wall time: 169 µs


In [9]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
dg_train, dg_valid, dg_test = data_generate(ds, lead_time)
model = train(dg_train, dg_valid, dg_test, model_save_fn)

[2024-03-26 23:58:38]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 


2024-03-26 23:58:58.078248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 64, 2)]       0         
                                                                 
 periodic_conv2d (PeriodicC  (None, 32, 64, 64)        3264      
 onv2D)                                                          
                                                                 
 tf.nn.elu (TFOpLambda)      (None, 32, 64, 64)        0         
                                                                 
 periodic_conv2d_1 (Periodi  (None, 32, 64, 64)        102464    
 cConv2D)                                                        
                                                                 
 tf.nn.elu_1 (TFOpLambda)    (None, 32, 64, 64)        0         
                                                                 
 periodic_conv2d_2 (Periodi  (None, 32, 64, 64)        102464

2024-03-26 23:58:59.898613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-03-26 23:59:02.118960: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14e2a75549e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 23:59:02.118979: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-03-26 23:59:02.250234: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 23:59:02.672123: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4655/4655 [==============================] - 78s 16ms/step - loss: 0.5080 - val_loss: 0.4751
Epoch 2/100
4655/4655 [==============================] - 72s 16ms/step - loss: 0.4549 - val_loss: 0.4522
Epoch 3/100
4655/4655 [==============================] - 72s 15ms/step - loss: 0.4369 - val_loss: 0.4433
Epoch 4/100
4655/4655 [==============================] - 72s 16ms/step - loss: 0.4256 - val_loss: 0.4359
Epoch 5/100
4655/4655 [==============================] - 73s 16ms/step - loss: 0.4169 - val_loss: 0.4289
Epoch 6/100
4655/4655 [==============================] - 72s 16ms/step - loss: 0.4099 - val_loss: 0.4258
Epoch 7/100
4655/4655 [==============================] - 73s 16ms/step - loss: 0.4042 - val_loss: 0.4196
Epoch 8/100
4655/4655 [==============================] - 73s 16ms/step - loss: 0.3994 - val_loss: 0.4219
Epoch 9/100
4655/4655 [==============================] - 73s 16ms/step - loss: 0.3951 - val_loss: 0.4138
Epoch 10/100
4655/4655 [==============================] - 73s 16ms/

In [10]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
pred = prediction(model, dg_test, iterative, iterative_lead_time, pred_save_fn) 
evaluate(pred, iterative, test_years)

[2024-03-27 00:22:14]
Create predictions ...
Saving predictions: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_cnn_3d.nc
Evaluating forecast ...
all test_years: [2021, 2022]
load_test_data, var: z
load_test_data, var: t
<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 640.4
    t        float64 2.934
CPU times: user 14.6 s, sys: 2.63 s, total: 17.2 s
Wall time: 13.7 s


# 120 hours (5 days)

In [11]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

print (60*"-")
batch_size = 32
vars = ('z', 't')
lead_time = 120
filters = [64, 64, 64, 64, 2]
kernels = [5, 5, 5, 5, 5]
dr = 0
patience = 3
save_prefix = 'NCI_tutorial' 
print ('save_prefix:', save_prefix)
model_save_fn= f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_cnn_5d.h5'
pred_save_fn = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_cnn_5d.nc'
lr = 1e-4
iterative = False
iterative_lead_time = None

[2024-03-27 00:22:28]
------------------------------------------------------------
save_prefix: NCI_tutorial
CPU times: user 90 µs, sys: 0 ns, total: 90 µs
Wall time: 96.8 µs


In [12]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
dg_train, dg_valid, dg_test =  data_generate(ds, lead_time)
model = train(dg_train, dg_valid, dg_test, model_save_fn)

[2024-03-27 00:22:28]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 64, 2)]       0         
                                                                 
 periodic_conv2d_5 (Periodi  (None, 32, 64, 64)        3264      
 cConv2D)                                                        
                                                                 
 tf.nn.elu_4 (TFOpLambda)    (None, 32, 64, 64)        0         
                                                     

In [13]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
pred = prediction(model, dg_test, iterative, iterative_lead_time, pred_save_fn) 
evaluate(pred, iterative, test_years)

[2024-03-27 00:36:18]
Create predictions ...
Saving predictions: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/NCI_tutorial_cnn_5d.nc
Evaluating forecast ...
all test_years: [2021, 2022]
load_test_data, var: z
load_test_data, var: t
<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 770.4
    t        float64 3.436
CPU times: user 14.5 s, sys: 2.17 s, total: 16.7 s
Wall time: 13 s


# fccnn_6h_iter

In [14]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

print (60*"-")
batch_size = 32
vars = ('z', 't')
lead_time = 6
iterative_lead_time = 5 * 24 
filters = [64, 64, 64, 64, 2]
kernels = [5, 5, 5, 5, 5]
dr = 0
patience = 3
save_prefix = 'NCI_tutorial' 
print ('save_prefix:', save_prefix)
model_save_fn=f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_fccnn_6h_iter.h5'
pred_save_fn =f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_fccnn_6h_iter.nc'
lr = 1e-4
iterative = True

[2024-03-27 00:36:31]
------------------------------------------------------------
save_prefix: NCI_tutorial
CPU times: user 0 ns, sys: 82 µs, total: 82 µs
Wall time: 89.4 µs


In [15]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
dg_train, dg_valid, dg_test = data_generate(ds, lead_time)
model = train(dg_train, dg_valid, dg_test, model_save_fn)

[2024-03-27 00:36:31]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 64, 2)]       0         
                                                                 
 periodic_conv2d_10 (Period  (None, 32, 64, 64)        3264      
 icConv2D)                                                       
                                                                 
 tf.nn.elu_8 (TFOpLambda)    (None, 32, 64, 64)        0         
                                                     

In [16]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
pred = prediction(model, dg_test, iterative, iterative_lead_time, pred_save_fn)  
evaluate(pred, iterative, test_years)

[2024-03-27 01:25:48]
Create predictions ...
Create iterative predictions
0
548/548 [==============================] - 4s 6ms/step
1
548/548 [==============================] - 3s 5ms/step
2
548/548 [==============================] - 3s 5ms/step
3
548/548 [==============================] - 3s 5ms/step
4
548/548 [==============================] - 3s 5ms/step
5
548/548 [==============================] - 3s 5ms/step
6
548/548 [==============================] - 3s 5ms/step
7
548/548 [==============================] - 3s 5ms/step
8
548/548 [==============================] - 3s 5ms/step
9
548/548 [==============================] - 3s 5ms/step
10
548/548 [==============================] - 3s 5ms/step
11
548/548 [==============================] - 3s 5ms/step
12
548/548 [==============================] - 3s 5ms/step
13
548/548 [==============================] - 3s 5ms/step
14
548/548 [==============================] - 3s 5ms/step
15
548/548 [==============================] - 3s 5ms/step
16
548/5

In [17]:
pred

<xarray.Dataset>
Dimensions:    (lead_time: 20, time: 17514, lat: 32, lon: 64)
Coordinates:
  * lead_time  (lead_time) int64 6 12 18 24 30 36 42 ... 90 96 102 108 114 120
  * time       (time) datetime64[ns] 2021-01-01 ... 2022-12-31T17:00:00
  * lat        (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon        (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Data variables:
    z          (lead_time, time, lat, lon) float32 4.857e+04 ... 5.058e+04
    t          (lead_time, time, lat, lon) float32 253.6 253.6 ... 259.5 259.5

In [18]:
client.shutdown()